# Instructions
In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.

1)Create a table rentals_may to store the data from rental table with information for the month of May.

2)Insert values in the table rentals_may using the table rental, filtering values only for the month of May.

3)Create a table rentals_june to store the data from rental table with information for the month of June.

4)Insert values in the table rentals_june using the table rental, filtering values only for the month of June.

5)Check the number of rentals for each customer for May.

6)Check the number of rentals for each customer for June.

#### 7)Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

8)Check the number of rentals for each customer for May

9)Check the number of rentals for each customer for June

Hint: You can store the results from the two queries in two separate dataframes.

10)Write a function that checks if customer borrowed more or less books in the month of June as compared to May.

Hint: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the merge function.

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = 'mlefnc85L63l682p'

In [2]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
data = pd.read_sql_query('SELECT * FROM sakila.rental_may', engine)
data.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [3]:
data.shape

(1183, 7)

In [4]:
#df1
query = "select customer_id, count(rental_id) from sakila.rental_may group by customer_id order by count(rental_id) desc;"

query_may = pd.read_sql_query(query, engine)
query_may.shape

(520, 2)

In [5]:
#df2
query2 = "select customer_id, count(rental_id) from sakila.rental_june group by customer_id order by count(rental_id) desc;"

query_june = pd.read_sql_query(query2, engine)
query_june.shape

(591, 2)

### 10)Write a function that checks if customer borrowed more or less film in the month of June as compared to May.
#data1=query_may.merge(query_june,left_on='customer_id',right_on='customer_id')
#data1.head()


In [19]:
#we create the categories for the output on the input:"Customer_id", if the count on the customers rentals in June is bigger
#than the rentals in May and we subtract to the rentals in May the rentals from June the result would be a negative number.
#A negative number can only correspond to a higher count compared to May [x]<0='more in June', on the opposite if May count 
#has a positive outcome [x]>0='more in May',else [x]=0 correspond to rented the same number of times.



def rental_info(df3,col):
    x=int(input("Customer_ID:"))
    if df3[col][x]>0:
        print("The customer {} rented more in May".format(x))
    elif df3[col][x]==0:
        print("The customer {} rented same amount in May and in June".format(x))
    elif df3[col][x]<0:
        print("The customer {} rented more in June".format(x))
  

## BIG PROBLEM

THE LAST CONDITION IS NOT COMPLETE!!! The function is not suffiecient to define the combination for whom rented 
in May or June as it does not distinguish among who rented more if in the merge there are Nan on empty cells!!!!!

I tried first with 'inner' merge but it exclude all the nan values, left join looked better as solution.

In [20]:
#check rentals 
#def rentals(df1,df2,how,on_column,col):#on_column,how =str
    #merge df1 and df2
df3=df1.merge(df2, how=how, on=on_column) 
    #rename columns
df3.columns=col
     
    #reseting index
df3=df3.set_index(col[0])
    
    #fill Nan values with 0 
df3.fillna(0, inplace=True) 
    
    #creating the difference
df3["diff"]=df3[col[1]]-df3[col[2]]
    
rental_info(df3,"diff")
    
    #return df3


In [14]:
col=['customer_id','rental_may','rental_june']


In [21]:
rentals(query_may,query_june,'outer','customer_id',col)

Customer_ID:31
The customer 31 rented more in June


,rental_may,rental_june,diff
customer_id,,,
197,8.0,8.0,0.0
506,7.0,5.0,2.0
109,7.0,5.0,2.0
269,6.0,3.0,3.0
239,6.0,7.0,-1.0
...,...,...,...
458,0.0,2.0,-2.0
487,0.0,2.0,-2.0
335,0.0,2.0,-2.0
